In [73]:
import plotly.express as px
import pandas as pd

In [74]:
generation_df = pd.read_csv('data/electricity_generation.csv')
consume_df = pd.read_csv('data/energy_consumption.csv')

In [75]:
generation_df.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,CLETPUS,194913,135451.32,1,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours
1,CLETPUS,195013,154519.994,1,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours
2,CLETPUS,195113,185203.657,1,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours
3,CLETPUS,195213,195436.666,1,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours
4,CLETPUS,195313,218846.325,1,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours


In [76]:
consume_df.head()

,MSN,YYYYMM,Value,Column_Order,Description,Unit
0,TXRCBUS,194913,4460.588,1,Primary Energy Consumed by the Residential Sector,Trillion Btu
1,TXRCBUS,195013,4829.528,1,Primary Energy Consumed by the Residential Sector,Trillion Btu
2,TXRCBUS,195113,5104.680,1,Primary Energy Consumed by the Residential Sector,Trillion Btu
3,TXRCBUS,195213,5158.406,1,Primary Energy Consumed by the Residential Sector,Trillion Btu
4,TXRCBUS,195313,5052.749,1,Primary Energy Consumed by the Residential Sector,Trillion Btu


In [77]:
generation_df = generation_df[generation_df['Value'] != generation_df.Value.value_counts().index[0] ]

In [78]:
def dataset_handling(df):
    df = df.drop(['MSN','Column_Order'], axis=1)
    df['Value'] = df.Value.astype('float64')
    df['YYYYMM'] = df.YYYYMM.astype('str')
    df['Month'] = df.YYYYMM.str[4:6]
    df['Year'] = df.YYYYMM.str[0:4]
    df['Type_Month'] = df['Month'].apply(lambda x: 'Normal_Months' if int(x) <13 else 'Month_13' )
    df['Year'] = df.Year.astype(int)
    df['Month'] = df.Month.astype(int)
    df = df[df['Month'] != 13]
    df = df.drop('Type_Month', axis=1)
    df['Date'] = df.YYYYMM.apply(lambda x: str(x[0:4]) + str('-') + str(x[4:]))
    df['Date'] = pd.to_datetime(df.Date, format= '%Y-%m')    
    
    return df 

    

In [79]:
generation = dataset_handling(generation_df)
consume = dataset_handling(consume_df)

In [80]:
consume.head()

,YYYYMM,Value,Description,Unit,Month,Year,Date
24,197301,1339.270,Primary Energy Consumed by the Residential Sector,Trillion Btu,1,1973,1973-01-01
25,197302,1174.899,Primary Energy Consumed by the Residential Sector,Trillion Btu,2,1973,1973-02-01
26,197303,983.374,Primary Energy Consumed by the Residential Sector,Trillion Btu,3,1973,1973-03-01
27,197304,715.391,Primary Energy Consumed by the Residential Sector,Trillion Btu,4,1973,1973-04-01
28,197305,535.914,Primary Energy Consumed by the Residential Sector,Trillion Btu,5,1973,1973-05-01


In [81]:
generation.head()

,YYYYMM,Value,Description,Unit,Month,Year,Date
24,197301,75190.149,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours,1,1973,1973-01-01
25,197302,67797.946,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours,2,1973,1973-02-01
26,197303,67387.612,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours,3,1973,1973-03-01
27,197304,63935.049,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours,4,1973,1973-04-01
28,197305,64927.181,"Electricity Net Generation From Coal, All Sectors",Million Kilowatthours,5,1973,1973-05-01


In [82]:
generation['Description'] = generation['Description'].replace(
    to_replace=[r"Electricity Net Generation From", r", All Sectors"], value=['ENRG', ''], regex=True)
generation.Description.unique()

consume['Description'] = consume.Description.replace(
    to_replace=['Total Energy Consumed', 'Primary Energy Consumed', 'by the ', ' Sector', 'Primary Energy Consumption'],
    value=['TEC', 'PEC', '', '', 'PE Consumption'], regex=True
)
consume.Description.unique()

array(['PEC Residential', 'TEC Residential', 'PEC Commercial',
       'TEC Commercial', 'PEC Industrial', 'TEC Industrial',
       'PEC Transportation', 'TEC Transportation', 'PEC Electric Power',
       'Energy Consumption Balancing Item', 'PE Consumption Total'],
      dtype=object)

In [83]:
consume_df.Description.unique()

array(['Primary Energy Consumed by the Residential Sector',
       'Total Energy Consumed by the Residential Sector',
       'Primary Energy Consumed by the Commercial Sector',
       'Total Energy Consumed by the Commercial Sector',
       'Primary Energy Consumed by the Industrial Sector',
       'Total Energy Consumed by the Industrial Sector',
       'Primary Energy Consumed by the Transportation Sector',
       'Total Energy Consumed by the Transportation Sector',
       'Primary Energy Consumed by the Electric Power Sector',
       'Energy Consumption Balancing Item',
       'Primary Energy Consumption Total'], dtype=object)

In [84]:
generation.head()

,YYYYMM,Value,Description,Unit,Month,Year,Date
24,197301,75190.149,ENRG Coal,Million Kilowatthours,1,1973,1973-01-01
25,197302,67797.946,ENRG Coal,Million Kilowatthours,2,1973,1973-02-01
26,197303,67387.612,ENRG Coal,Million Kilowatthours,3,1973,1973-03-01
27,197304,63935.049,ENRG Coal,Million Kilowatthours,4,1973,1973-04-01
28,197305,64927.181,ENRG Coal,Million Kilowatthours,5,1973,1973-05-01


In [85]:
def gen_merged_sector(sector_consumed, sector_produced):
    con_df = consume[consume['Description'] == sector_consumed].copy()
    gen_df = generation[generation['Description'] == sector_produced].copy()

    con_df = con_df.drop(['Description', 'Unit', 'Month', 'Year', 'Date'], axis=1)
    gen_df = gen_df.drop(['Description', 'Unit'], axis=1)

    merged = con_df.merge(gen_df, on= 'YYYYMM')
    merged = merged.rename(columns= {'Value_x':'Consumed', 'Value_y':'Generated'})
    
    return merged


electric_nuclear = gen_merged_sector('PEC Electric Power', 'ENRG Nuclear Electric Power')
electric_nuclear

,YYYYMM,Consumed,Generated,Month,Year,Date
0,197301,1691.096,6246.251,1,1973,1973-01-01
1,197302,1511.458,5928.069,2,1973,1973-02-01
2,197303,1559.159,6649.007,3,1973,1973-03-01
3,197304,1470.152,5876.392,4,1973,1973-04-01
4,197305,1551.631,5696.657,5,1973,1973-05-01
...,...,...,...,...,...,...
557,201906,3180.175,68804.879,6,2019,2019-06-01
558,201907,3701.225,72198.595,7,2019,2019-07-01
559,201908,3601.167,71910.684,8,2019,2019-08-01
560,201909,3230.691,65892.180,9,2019,2019-09-01


In [86]:
m_consume = consume.drop(['Unit', 'Month', 'Year', 'Date'], axis=1)
m_generation = generation.drop(['Unit'], axis=1)
gen_cons_energy = m_consume.merge(m_generation, on="YYYYMM")
gen_cons_energy = gen_cons_energy.rename(columns={'Value_x':'Consumed', 'Value_y':'Generated',
                                                  'Description_x':'Description_Cons', 
                                                  'Description_y':'Description_Gen'})
gen_cons_energy


,YYYYMM,Consumed,Description_Cons,Generated,Description_Gen,Month,Year,Date
0,197301,1339.270,PEC Residential,75190.149,ENRG Coal,1,1973,1973-01-01
1,197301,1339.270,PEC Residential,31183.619,ENRG Petroleum,1,1973,1973-01-01
2,197301,1339.270,PEC Residential,21184.967,ENRG Natural Gas,1,1973,1973-01-01
3,197301,1339.270,PEC Residential,6246.251,ENRG Nuclear Electric Power,1,1973,1973-01-01
4,197301,1339.270,PEC Residential,26249.203,ENRG Conventional Hydroelectric Power,1,1973,1973-01-01
...,...,...,...,...,...,...,...,...
73233,201910,7976.769,PE Consumption Total,1565.985,ENRG Waste,10,2019,2019-10-01
73234,201910,7976.769,PE Consumption Total,1276.179,ENRG Geothermal,10,2019,2019-10-01
73235,201910,7976.769,PE Consumption Total,6109.899,ENRG Solar,10,2019,2019-10-01
73236,201910,7976.769,PE Consumption Total,28143.676,ENRG Wind,10,2019,2019-10-01


In [87]:
gen_cons_energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73238 entries, 0 to 73237
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   YYYYMM            73238 non-null  object        
 1   Consumed          73238 non-null  float64       
 2   Description_Cons  73238 non-null  object        
 3   Generated         73238 non-null  float64       
 4   Description_Gen   73238 non-null  object        
 5   Month             73238 non-null  int32         
 6   Year              73238 non-null  int32         
 7   Date              73238 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int32(2), object(3)
memory usage: 3.9+ MB


In [88]:
electric_nuclear

,YYYYMM,Consumed,Generated,Month,Year,Date
0,197301,1691.096,6246.251,1,1973,1973-01-01
1,197302,1511.458,5928.069,2,1973,1973-02-01
2,197303,1559.159,6649.007,3,1973,1973-03-01
3,197304,1470.152,5876.392,4,1973,1973-04-01
4,197305,1551.631,5696.657,5,1973,1973-05-01
...,...,...,...,...,...,...
557,201906,3180.175,68804.879,6,2019,2019-06-01
558,201907,3701.225,72198.595,7,2019,2019-07-01
559,201908,3601.167,71910.684,8,2019,2019-08-01
560,201909,3230.691,65892.180,9,2019,2019-09-01


In [89]:
px.line(electric_nuclear, x='Date', y='Consumed')



In [90]:
electric_nuclear['Consume_ratio'] = electric_nuclear.Consumed / electric_nuclear.Generated
electric_nuclear

,YYYYMM,Consumed,Generated,Month,Year,Date,Consume_ratio
0,197301,1691.096,6246.251,1,1973,1973-01-01,0.270738
1,197302,1511.458,5928.069,2,1973,1973-02-01,0.254966
2,197303,1559.159,6649.007,3,1973,1973-03-01,0.234495
3,197304,1470.152,5876.392,4,1973,1973-04-01,0.250179
4,197305,1551.631,5696.657,5,1973,1973-05-01,0.272376
...,...,...,...,...,...,...,...
557,201906,3180.175,68804.879,6,2019,2019-06-01,0.046220
558,201907,3701.225,72198.595,7,2019,2019-07-01,0.051265
559,201908,3601.167,71910.684,8,2019,2019-08-01,0.050078
560,201909,3230.691,65892.180,9,2019,2019-09-01,0.049030


In [91]:
electric_nuclear.to_csv('data/electric_nuclear.csv', sep=',')

In [92]:
a = electric_nuclear.groupby(by='Year').apply(lambda x: x.loc[x['Consume_ratio'].idxmax(), 
                                                              ['Generated', 'Consumed', 'Month', 'Consume_ratio']])
a

,Generated,Consumed,Month,Consume_ratio
Year,,,,
1973,5696.657,1551.631,5,0.272376
1974,6562.335,1630.562,5,0.248473
1975,12743.820,1723.312,6,0.135227
1976,10981.808,1594.553,4,0.145199
1977,21824.835,2131.452,7,0.097662
1978,17579.902,1700.153,4,0.096710
1979,15024.851,1881.371,5,0.125217
1980,21023.937,2318.950,7,0.110300
1981,21165.626,2172.290,6,0.102633


In [93]:
electric_nuclear[electric_nuclear['Year'] == 1973]

,YYYYMM,Consumed,Generated,Month,Year,Date,Consume_ratio
0,197301,1691.096,6246.251,1,1973,1973-01-01,0.270738
1,197302,1511.458,5928.069,2,1973,1973-02-01,0.254966
2,197303,1559.159,6649.007,3,1973,1973-03-01,0.234495
3,197304,1470.152,5876.392,4,1973,1973-04-01,0.250179
4,197305,1551.631,5696.657,5,1973,1973-05-01,0.272376
5,197306,1709.752,6784.152,6,1973,1973-06-01,0.252021
6,197307,1846.331,6960.486,7,1973,1973-07-01,0.265259
7,197308,1889.873,7785.326,8,1973,1973-08-01,0.242748
8,197309,1664.842,7862.383,9,1973,1973-09-01,0.211748
9,197310,1638.807,7518.195,10,1973,1973-10-01,0.217979


In [ ]:
pd.DataFrame.loc[]